In [1]:
from utils.data_preprocessing import *

In [2]:
base_path = os.path.join('..', 'data', 'dlis_preprocessed')
csv_data = load_csv_files(base_path)

In [3]:
rhob_logs = create_df_subset(csv_data, ['RHOB'])

In [4]:
remove_nan_values(rhob_logs)

c:\Users\rafae\Documents\College\PIBIC_Imagens\lithofacies-classification-from-image-logs\src\utils\data_preprocessing.py:315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
c:\Users\rafae\Documents\College\PIBIC_Imagens\lithofacies-classification-from-image-logs\src\utils\data_preprocessing.py:315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
c:\Users\rafae\Documents\College\PIBIC_Imagens\lithofacies-classification-from-image-logs\src\utils\data_preprocessing.py:315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

In [6]:
spliced_rhob_data = {}
for well, w_dict in rhob_logs.items():
    well_data = []

    for logical_file, lf_dict in w_dict.items():
        for frame, df in lf_dict.items():
            if 'RHOB' in df.columns:
                well_data.append(df)

    try:
        df_concat = pd.concat(well_data).sort_values(by='TDEP', ascending=True)
    except Exception as e:
        print(f"Error when concating {well}: {e}")
        df_concat = None

    spliced_rhob_data[well] = df_concat

Error when concating 1IDA 0001  SE: No objects to concatenate
Error when concating 1MO  0001  SE: No objects to concatenate


In [8]:
spliced_dfs_to_csv(spliced_rhob_data, '../data/dlis_spliced/RHOB')

Error creating the CSV file for the well 1IDA 0001  SE: 'NoneType' object has no attribute 'to_csv'
Error creating the CSV file for the well 1MO  0001  SE: 'NoneType' object has no attribute 'to_csv'
Error creating the CSV file for the well 1IDA 0001  SE: 'NoneType' object has no attribute 'to_csv'
